In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from collections import OrderedDict

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from keybert import KeyBERT

In [3]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import matplotlib.pyplot as plt
import spacy
import string
from sklearn.metrics.pairwise import cosine_similarity

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
#list(gensim.downloader.info()['models'].keys())
#wv = api.load('glove-wiki-gigaword-300')
#wv.save('glove_wiki_vectors.kv')

In [5]:
revalive_path_vec = "../../glove_wiki_vectors.kv"
relative_path_sdg = "../../sdg_data.csv"
relative_path_attr = "../Dictionary/Attributes_2019-20.csv"
relative_string_map = "./Data_Goal/String_map.csv"
relative_string_map_unroll = "./Data_Goal/String_map_unroll.csv"
relative_semantic = "./Data_Goal/Semantic_map.csv"
relative_semantic_unroll = "./Data_Goal/Semantic_map_unroll.csv"
relative_sem_threshold = "./Data_Goal/Semantic_threshold_map.csv"
relative_sem_threshold_unroll = "./Data_Goal/Semantic_threshold_map_unroll.csv"

# Get the absolute path by joining the current directory with the relative path
absolute_path_vec = os.path.normpath(os.path.join(os.getcwd(), revalive_path_vec))
absolute_path_sdg = os.path.normpath(os.path.join(os.getcwd(), relative_path_sdg))
absolute_path_attr = os.path.normpath(os.path.join(os.getcwd(), relative_path_attr))
absolute_path_string = os.path.normpath(os.path.join(os.getcwd(), relative_string_map))
absolute_path_string_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_string_map_unroll))
absolute_path_semantic = os.path.normpath(os.path.join(os.getcwd(), relative_semantic))
absolute_path_semantic_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_semantic_unroll))
absolute_path_sem_threshold =os.path.normpath( os.path.join(os.getcwd(), relative_sem_threshold))
absolute_path_sem_threshold_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_sem_threshold_unroll))

In [6]:
wv = KeyedVectors.load(absolute_path_vec)

In [7]:
kw_model = KeyBERT(model=wv)

In [8]:
def replace_words(df, column_list, replace_dict):
    for col in column_list:
        # Apply the replace function to the specified column using the replace_dict
        df[col] = df[col].replace(replace_dict, regex=True)
    return df

# Define the replacement dictionary
replace_dict = {
    'B.p.l.': 'below poverty line',
    'A.p.l.': 'above poverty line',
    'Bpl': 'below poverty line',
    'Fisheries' : 'Fisheries and Animal Husbandry',
    'Holder' : '(Below poverty line / Above poverty line)'
}


In [9]:
IGNORE_WORDS = ['proportion', 'whose', 'number', 'per', 'within', 'no', 'under', 'volume', 'use', 'type', 'percentage', 'total', 'data', 'year', 'due', 'rate', 'hects', 'hect', 'noof', 'years']

In [10]:
# Creating our tokenizer function
def cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()
    ignore_word = []
    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        #letters_list = list(set(letters))
        letters_list = list(OrderedDict.fromkeys(letters))   
        cleaned.append(letters_list)

    return cleaned

In [11]:
# Creating our tokenizer function
def concat_cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()

    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        #letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        #IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        #letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        letters = ' '.join(letters)
        cleaned.append(letters)

    return cleaned

In [12]:
def process_keyword(data):
    result = []
    for sublist in data:
        inner_list = []
        for word, _ in sublist:
            # Split the word if it contains a space and reverse the order of words
            words = word.split()
            # Filter out numeric values
            words = [w for w in words if not w.isdigit()]
            if words:  # Exclude empty lists
                inner_list.append(words)
        inner_list.reverse()
        result.append(inner_list)
    return result

In [13]:
def add_elements(row):
    if not row['keywords']:  # Check if col2 is empty
        row['keywords'].extend([[elem] for elem in row['new_goal']])  # Add elements from col1 to col2 as lists
    return row

In [14]:
def concatenate_table_name(df):
    for index, row in df.iterrows():
        if ',' not in row['Description']:
            df.at[index, 'Description'] += ', ' + row['Table_name']
        elif row['Description'].count(',') == 1:
            parts = row['Description'].split(',')
            if len(parts) == 2:
                part1, part2 = parts
                if re.match(r'^\w+$', part1.strip()) or re.match(r'^\w+$', part2.strip()):
                    df.at[index, 'Description'] += ', ' + row['Table_name']
    return df


In [15]:
def concatenate_all(df):
    for index, row in df.iterrows():
        df.at[index, 'Description'] += ', ' + row['Table_name'] + ', ' + row['Chapter_name'] 
    return df

In [16]:
missing_words = []
miss_added = []
def sent_vec(row):
    sent = row['new_description']
    cap_tab = row['Chapter_name'] + ", " + row['Table_name']
    vector_size = wv.vector_size
    vectors = []
    miss_w = []
    for w in sent:
        if w in wv:
            wv_res = np.zeros(vector_size)
            wv_res = wv[w]
            vectors.append(wv_res)
        else:
            missing_words.append(("KAG", w))
            miss_w.append(w)
    if len(miss_w)>0:
        vectors.append(missin_word_vec(cap_tab))
        miss_added.append(miss_w)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [17]:
def missin_word_vec(sent):
    vector_size = wv.vector_size
    sent_list = cleaning([sent])
    wv_res = np.zeros(vector_size)
    ctr = 0
    for w in sent_list[0]:
        if w in wv:
            wv_res += wv[w]
            ctr += 1
    if ctr>0:
        wv_res = wv_res/ctr
    else:
        wv_res = np.random.rand(vector_size)
        print('yes')
    return wv_res

In [18]:
def sent_vec_avg(sent):
    vector_size = wv.vector_size
    missing_words = []
    vectors = []
    for w in sent:
        ctr = 0
        wv_res = np.zeros(vector_size)
        for word in  w:
            if word in wv:
                wv_res += wv[word]
                ctr += 1
            else:
                missing_words.append(word)
        if ctr>0:
            wv_res = wv_res/ctr
        vectors.append(wv_res)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [19]:
def count_common_words(row, df_other):
    set_1 = set(row)
    common_words = []
    for _, other_row in df_other.iterrows():
        set_2 = set(other_row['new_description'])
        common_words.append(len(set_1.intersection(set_2)))
    return common_words

In [20]:
def find_similarity_string(similarity_matrix, threshold, chapter_df, sdg_df):
    top_n_similarities = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_chapter_name = []
    top_n_table_id = []
    top_n_table_name = []
    top_n_count = []
    top_n_description = []
    
    for index, row in similarity_matrix.iterrows():
        # Filter indices based on threshold value
        top_n_indices = [i for i, value in enumerate(row) if value >= threshold]
        
        # Sort indices based on similarity values
        top_n_indices = sorted(top_n_indices, key=lambda i: row.iloc[i], reverse=True)
        
        # Get top n values, chapters, and chapter names
        values = [row.iloc[i] for i in top_n_indices]
        attr = [similarity_matrix.columns[i] for i in top_n_indices]
        attr_des = [chapter_df['Description'].loc[i] for i in top_n_indices]
        chapter_id = [chapter_df['Chapter_id'].loc[i] for i in top_n_indices]
        chapter_name = [chapter_df['Chapter_name'].loc[i] for i in top_n_indices]   
        table_id = [chapter_df['Table_id'].loc[i] for i in top_n_indices]
        table_name = [chapter_df['Table_name'].loc[i] for i in top_n_indices]   


        # Append to respective lists
        top_n_count.append(len(attr))
        top_n_similarities.append(values)
        top_n_attr.append(attr)
        top_n_description.append(attr_des)
        top_n_chapter_id.append(chapter_id)
        top_n_chapter_name.append(chapter_name)
        top_n_table_id.append(table_id)
        top_n_table_name.append(table_name)
    
    # Assign lists to the SDG DataFrame
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_table_name'] = top_n_table_name
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_similarities'] = top_n_similarities
    sdg_df['top_n_description'] = top_n_description

    return sdg_df


In [21]:
def process_result_string(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        values = row['top_n_similarities']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        
        for attr, chap, tab, val, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, values, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no,
                'top_n_similarities': val
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'top_n_goal_id', 'top_n_similarities']]
    return all_goals


In [22]:
def find_similarity_semantic(sdg_df, att_df, threshold):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_similarity_values = []
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        top_min_list = []
        for i in ind_list[ind]:
            max_list = []
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_similarity = np.max(similarity_matrix)
                max_list.append(max_similarity)
            # Get the indices of the top 10 max values
            #top_indices = [idx for idx, sim in enumerate(max_list) if sim < 1]
            top_indices = sorted(range(len(max_list)), key=lambda i: max_list[i], reverse=True)[:threshold]
            top_indices_list.append(top_indices)
            top_sim = [max_list[i] for i in top_indices]
            top_sim_list.append(top_sim)
            top_min_list.append(min(top_sim))
            
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
            
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        min_similarity_values.append(top_min_list)
    return min_similarity_values, top_indices_values


In [23]:
#indicator words to attribute complete name
def find_similarity_semantic_threshold(sdg_df, att_df, diff):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_list = sdg_df['min_values'].tolist()
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        min_values = min_list[ind]
        for index, i in enumerate(ind_list[ind]):
            max_list = []
            threshold = min_values[index] - diff
            if(threshold <= 0):
                continue
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_value = np.max(similarity_matrix)
                max_list.append((max_value, att))
            
            # Get the indices of values strictly greater than threshold
            top_indices = [idx for val, idx in max_list if val > threshold]
            # Sort top_indices based on the corresponding values in max_list
            top_indices.sort(key=lambda x: next((val for val, idx in max_list if idx == x), 0), reverse=True)
            top_indices_list.append(top_indices)
            top_sim = [max_list[i][0] for i in top_indices]
            top_sim_list.append(top_sim)
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
        
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        
    return top_indices_values

In [24]:
def finding_attr_details(sdg_df, att_df):
    result_list = sdg_df['top_n_index'].tolist()
    top_n_count = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_table_id = []
    top_n_description = []
    top_n_chapter_name = []
    top_n_table_name = []
    for ll in result_list:
        top_n_count.append(len(ll))
        top_n_attr.append([att_df.loc[idx, 'Attr_id'] for idx in ll])
        top_n_chapter_id.append([att_df.loc[idx, 'Chapter_id'] for idx in ll])
        top_n_table_id.append([att_df.loc[idx, 'Table_id'] for idx in ll])
        top_n_description.append([att_df.loc[idx, 'Description'] for idx in ll])
        top_n_chapter_name.append([att_df.loc[idx, 'Chapter_name'] for idx in ll])
        top_n_table_name.append([att_df.loc[idx, 'Table_name'] for idx in ll])
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_description'] = top_n_description
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_table_name'] = top_n_table_name
    return sdg_df

In [25]:
def process_result_semantic(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        
        for attr, chap, tab, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'top_n_goal_id']]
    return all_goals


In [26]:
sdg = pd.read_csv(absolute_path_sdg, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [27]:
sdg_complete = sdg.copy()

In [28]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [29]:
sdg.columns

Index(['Goal No.', 'Goal', 'Nodal Department', 'Targets',
       'Other Related Major Departments', 'Tentative Indicators', 'Target_id',
       'Indicator_id'],
      dtype='object')

In [30]:
sdg.drop(['Nodal Department','Other Related Major Departments', 'Tentative Indicators', 'Indicator_id', 'Targets', 'Target_id'],axis=1,inplace=True)

In [31]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [32]:
sdg.reset_index(inplace = True, drop = True)

In [33]:
sdg

,Goal No.,Goal
0,1,End poverty in all its forms everywhere
1,2,"End hunger, achieve food security and improved..."
2,3,Ensure healthy lives and promote well-being fo...
3,4,Ensure inclusive and equitable quality educati...
4,5,Achieve gender equality and empower all women ...
5,6,Ensure availability and sustainable management...
6,7,"Ensure access to affordable, reliable, sustain..."
7,8,"Promote sustained, inclusive and sustainable e..."
8,9,"Build resilient infrastructure, promote inclus..."
9,10,Reduce inequality within the State


In [34]:
sdg_new = sdg.copy()

In [35]:
texto_data=sdg_new['Goal'].tolist()
process_text=cleaning(texto_data)

cleaned_sent = pd.DataFrame({'new_goal': process_text})
sdg_new['new_goal']=cleaned_sent

In [36]:
texto_data0=sdg_new['Goal'].tolist()
process_text0=concat_cleaning(texto_data0)

cleaned_sent0 = pd.DataFrame({'new_goal_str': process_text0})
sdg_new['new_goal_str']=cleaned_sent0

In [37]:
sdg_new

,Goal No.,Goal,new_goal,new_goal_str
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state


In [38]:
data = pd.read_csv(absolute_path_attr, sep=";", dtype={'Attr_id': str, 'Chapter_id': str, 'Table_id':str})

In [39]:
# Replace words in the DataFrame using the dictionary
data = replace_words(data, ['Chapter_name', 'Table_name', 'Description'], replace_dict)

In [40]:
data_new = concatenate_all(data)
data_new

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Taluks, Taluks Hoblies Grama Panchayath Villag..."
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Hoblies, Taluks Hoblies Grama Panchayath Villa..."
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Grama Panchayaths, Taluks Hoblies Grama Pancha..."
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Circles, Taluks Hoblies Grama Panchayath Villa..."
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages, Inhabited Un-..."
...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019, Regi..."
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019, Regis..."
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019, Reg..."
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019..."


In [44]:
# Create a new DataFrame with selected columns
new_df = data_new.copy()
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Taluks, Taluks Hoblies Grama Panchayath Villag..."
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Hoblies, Taluks Hoblies Grama Panchayath Villa..."
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Grama Panchayaths, Taluks Hoblies Grama Pancha..."
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Circles, Taluks Hoblies Grama Panchayath Villa..."
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages, Inhabited Un-..."
...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019, Regi..."
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019, Regis..."
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019, Reg..."
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019..."


In [45]:
texto_data1=new_df['Description'].tolist()
process_text1=cleaning(texto_data1)

cleaned_sent1 = pd.DataFrame({'new_description': process_text1})
new_df['new_description']=cleaned_sent1

In [46]:
texto_data11=new_df['Description'].tolist()
process_text11=concat_cleaning(texto_data11)

cleaned_sent11 = pd.DataFrame({'new_description_str': process_text11})
new_df['new_description_str']=cleaned_sent11

In [47]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Taluks, Taluks Hoblies Grama Panchayath Villag...","[taluks, hoblies, grama, panchayath, village, ...",taluks taluks hoblies grama panchayath village...
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Hoblies, Taluks Hoblies Grama Panchayath Villa...","[hoblies, taluks, grama, panchayath, village, ...",hoblies taluks hoblies grama panchayath villag...
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Grama Panchayaths, Taluks Hoblies Grama Pancha...","[grama, panchayaths, taluks, hoblies, panchaya...",grama panchayaths taluks hoblies grama panchay...
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Circles, Taluks Hoblies Grama Panchayath Villa...","[circle, taluks, hoblies, grama, panchayath, v...",circles taluks hoblies grama panchayath villag...
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages, Inhabited Un-...","[un, inhabited, village, census, general, info...",un inhabited inhabited villages inhabited un i...
...,...,...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019, Regi...","[birth, late, registered, event, registration,...",total births late registered events registrati...
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019, Regis...","[male, death, late, registered, event, registr...",male deaths late registered events registratio...
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019, Reg...","[female, death, late, registered, event, regis...",female deaths late registered events registrat...
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019...","[transgender, death, late, registered, event, ...",transgender deaths late registered events regi...


String Matching

In [48]:
common_word_matrix = sdg_new['new_goal'].apply(count_common_words, args=(new_df,))
common_word_matrix

0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
10    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
11    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
12    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
13    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
14    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
15    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
16    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: new_goal, dtype: object

In [49]:
attr_id = new_df['Attr_id'].tolist()
ind_id = sdg_new['Goal No.'].tolist()

In [50]:
# Determine the maximum number of words in any list
max_words = max(common_word_matrix, key=lambda x: len(x))

# Create DataFrame with columns Word_1, Word_2, ..., Word_n
common_word_df = pd.DataFrame(common_word_matrix.tolist(), columns=attr_id, index=ind_id)

In [51]:
common_word_df

,3,4,5,6,7,8,9,10,11,12,...,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Example usage:
threshold_value = 1  # Define the threshold value

# Call the function to find chapter_ids with similarity greater than the threshold
sdg_string_map = find_similarity_string(common_word_df, threshold_value, new_df, sdg_new.copy())


In [53]:
sdg_string_map

,Goal No.,Goal,new_goal,new_goal_str,top_n_count,top_n_attr,top_n_table_id,top_n_table_name,top_n_chapter_id,top_n_chapter_name,top_n_similarities,top_n_description
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,250,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, ...",[Ration Shops and Ration Card (Below poverty l...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General Information, General Information, Gen...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Urban,Ration shops, Ration Shops and Ration C..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,500,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[4.3, 4.3, 4.6, 4.6, 4.12, 4.1, 4.1, 4.1, 4.1,...",[Area Under Principal crops in Hectares 2018-1...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[Agriculture, Horticulture & Sericulture, Agri...","[2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Total Food Grains,Pulses, Area Under Principa..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,36,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.1...",[Age wise Population per 2011 Census and Proje...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[Area and Population, Area and Population, Are...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Male,0-14 2011, Age wise Population per 2011 ..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,128,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[10.1, 10.1, 10.1, 10.1, 10.1, 10.1, 10.1, 10....","[Literates 2011 Census, Literates 2011 Census,...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[Education, Education, Education, Education, E...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Male,Rural, Literates 2011 Census, Education,..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,96,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[13.5, 10.3, 10.4, 10.5, 10.8, 10.8, 10.9, 10....",[No. of Anganawadi Teachers Workers and Supple...,"[13, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[Women & Child Development, Education, Educati...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Adolescent Girls,No. of Beneficiaries under S..."
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,30,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1.5, 9.3, 10.9, 10.10, 14.4, 14.4, 14.4, 14.4...","[No. of Fire Stations on 31-3-2020, Length of ...","[1, 9, 10, 10, 14, 14, 14, 14, 14, 14, 14, 14,...","[General Information, Transport and Communicat...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Working Fire Brigade Water Tankers, No. of Fi..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,51,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[9.3, 9.3, 9.3, 9.3, 9.3, 9.3, 9.3, 15.2, 16.1...",[Length of Rural Roads and Number of Habitatio...,"[9, 9, 9, 9, 9, 9, 9, 15, 16, 16, 16, 16, 16, ...","[Transport and Communication, Transport and Co...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Pucca Road,Panchayat Roads,Length of Rural Ro..."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,17,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[5.4, 

In [54]:
string_all_goals = process_result_string(sdg_string_map, sdg_new)

In [55]:
sdg_string_map.to_csv(absolute_path_string, sep=";", index=False)

In [56]:
string_all_goals.to_csv(absolute_path_string_unroll, sep=";", index=False)

Semantic Mapping

In [57]:
sdg_sem = sdg_new.copy()

In [58]:
word_count = sdg_sem['new_goal'].apply(lambda x: len(x))
word_count

0      4
1     10
2      6
3      9
4      6
5      6
6      7
7     11
8     10
9      3
10     8
11     5
12     7
13    17
14    14
15     8
16     2
Name: new_goal, dtype: int64

In [59]:
from collections import Counter
w_c = Counter(word_count)
w_c.most_common()

[(6, 3),
 (10, 2),
 (7, 2),
 (8, 2),
 (4, 1),
 (9, 1),
 (11, 1),
 (3, 1),
 (5, 1),
 (17, 1),
 (14, 1),
 (2, 1)]

In [60]:
w_c.most_common(10)

[(6, 3),
 (10, 2),
 (7, 2),
 (8, 2),
 (4, 1),
 (9, 1),
 (11, 1),
 (3, 1),
 (5, 1),
 (17, 1)]

In [61]:
median_indicator = np.median(word_count)
print(median_indicator)

7.0


In [62]:
from statistics import mode
mode_indicator = mode(word_count)
mode_indicator

6

In [63]:
np.mean(word_count)

7.823529411764706

In [64]:
stopwords_set = list(set(nltk.corpus.stopwords.words('english')))
stopwords_set.extend(IGNORE_WORDS)
stopwords_set = list(set(stopwords_set))
stopwords_set

['her',
 'or',
 "wouldn't",
 'are',
 'only',
 'did',
 "shouldn't",
 'hects',
 'any',
 'before',
 "won't",
 'yourself',
 'should',
 'if',
 'percentage',
 'and',
 'this',
 'same',
 'of',
 'didn',
 'nor',
 'type',
 'up',
 'myself',
 'you',
 'very',
 'weren',
 'rate',
 'out',
 'these',
 'its',
 'where',
 "weren't",
 'to',
 'now',
 'hadn',
 "you're",
 'below',
 'own',
 'within',
 'don',
 "didn't",
 'hasn',
 'd',
 'he',
 'whom',
 "doesn't",
 'does',
 'yourselves',
 'she',
 'from',
 'hect',
 'due',
 'until',
 'my',
 'had',
 'those',
 'why',
 'what',
 'there',
 'couldn',
 'use',
 'all',
 'some',
 'just',
 'am',
 'him',
 'few',
 'y',
 'doing',
 'number',
 'herself',
 'noof',
 'o',
 'on',
 'haven',
 'through',
 'having',
 "you'd",
 'been',
 'year',
 "that'll",
 'shouldn',
 'can',
 'how',
 'that',
 'were',
 'a',
 'between',
 'at',
 'm',
 'being',
 'our',
 'has',
 'wouldn',
 'down',
 'whose',
 'an',
 'shan',
 'so',
 'per',
 'not',
 'total',
 'against',
 'we',
 'themselves',
 'for',
 'such',
 'need

In [65]:
#using 'new_indicators_str', takin top_n=6 as median = 8 and nr_candidates=10 as mean = 9.2
sdg_sem['keywords']=kw_model.extract_keywords(sdg_sem['new_goal_str'].tolist(), keyphrase_ngram_range=(1,2), stop_words = stopwords_set, use_maxsum=True, nr_candidates=10, top_n=6)

In [66]:
sdg_sem['keywords'] = process_keyword(sdg_sem['keywords'].tolist())
sdg_sem

,Goal No.,Goal,new_goal,new_goal_str,keywords
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge..."
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,"[[modern, energy], [ensure], [affordable, reli..."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,"[[full, productive], [promote, sustained], [de..."
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...,"[[promote], [inclusive, sustainable], [foster,..."
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state,[]


In [67]:
# Apply the function to each row of the DataFrame
sdg_sem = sdg_sem.apply(add_elements, axis=1)
sdg_sem

,Goal No.,Goal,new_goal,new_goal_str,keywords
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge..."
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,"[[modern, energy], [ensure], [affordable, reli..."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,"[[full, productive], [promote, sustained], [de..."
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...,"[[promote], [inclusive, sustainable], [foster,..."
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state,"[[reduce], [inequality], [state]]"


In [68]:
sdg_sem['vec'] = sdg_sem['keywords'].apply(sent_vec_avg)
sdg_sem

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [...","[[0.004565000534057617, 0.3802250027656555, 0...."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv...","[[0.07902299799025059, 0.040918700862675905, -..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h...","[[-0.13508999347686768, 0.3590700030326843, 0...."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l...","[[-0.15599000453948975, -0.4886299967765808, -..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge...","[[-0.4389500021934509, 0.47641998529434204, 0...."
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m...","[[0.2156050056219101, -0.03203999996185303, 0...."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,"[[modern, energy], [ensure], [affordable, reli...","[[0.4583650156855583, -0.1715994980186224, -0...."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,"[[full, productive], [promote, sustained], [de...","[[-0.29617999494075775, 0.21565200434997678, -..."
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...,"[[promote], [inclusive, sustainable], [foster,...","[[0.0451820008456707, 0.06997399777173996, -0...."
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state,"[[reduce], [inequality], [state]]","[[0.21908999979496002, 0.35475999116897583, 0...."


In [69]:
new_df['vec'] = new_df.apply(sent_vec, axis=1)

In [70]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str,vec
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Taluks, Taluks Hoblies Grama Panchayath Villag...","[taluks, hoblies, grama, panchayath, village, ...",taluks taluks hoblies grama panchayath village...,"[[-0.18463000655174255, -1.0946999788284302, -..."
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Hoblies, Taluks Hoblies Grama Panchayath Villa...","[hoblies, taluks, grama, panchayath, village, ...",hoblies taluks hoblies grama panchayath villag...,"[[-0.18463000655174255, -1.0946999788284302, -..."
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Grama Panchayaths, Taluks Hoblies Grama Pancha...","[grama, panchayaths, taluks, hoblies, panchaya...",grama panchayaths taluks hoblies grama panchay...,"[[0.09846500307321548, -0.4200499951839447, -0..."
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Circles, Taluks Hoblies Grama Panchayath Villa...","[circle, taluks, hoblies, grama, panchayath, v...",circles taluks hoblies grama panchayath villag...,"[[0.15791000425815582, 0.24285000562667847, -0..."
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages, Inhabited Un-...","[un, inhabited, village, census, general, info...",un inhabited inhabited villages inhabited un i...,"[[0.44261, -0.17972, -0.27015, -0.39474, -0.03..."
...,...,...,...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019, Regi...","[birth, late, registered, event, registration,...",total births late registered events registrati...,"[[-0.11292, -0.33504, -0.24223, 0.2406, -0.285..."
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019, Regis...","[male, death, late, registered, event, registr...",male deaths late registered events registratio...,"[[-0.32418, 0.4799, -0.17283, -0.2947, -0.0892..."
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019, Reg...","[female, death, late, registered, event, regis...",female deaths late registered events registrat...,"[[-0.2299, 0.42912, -0.25127, -0.19251, -0.262..."
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019...","[transgender, death, late, registered, event, ...",transgender deaths late registered events regi...,"[[-0.058667, -0.21876, 0.050555, 0.34848, -0.1..."


In [71]:
missing_words_set = list(set(missing_words))
print(len(missing_words_set), missing_words_set)

85 [('KAG', 'employement'), ('KAG', 'nrlm'), ('KAG', 'delivaries'), ('KAG', 'mgnrega'), ('KAG', 'sapota'), ('KAG', 'assoistance'), ('KAG', 'marketin'), ('KAG', 'multiaxled'), ('KAG', 'mythri'), ('KAG', 'teachersal'), ('KAG', 'hepatities'), ('KAG', 'socities'), ('KAG', 'areaunder'), ('KAG', 'estp'), ('KAG', 'nddp'), ('KAG', 'borewells'), ('KAG', 'avare'), ('KAG', 'brambedkar'), ('KAG', 'pomogranate'), ('KAG', 'priva'), ('KAG', 'airculated'), ('KAG', 'raingauge'), ('KAG', 'tubectomy'), ('KAG', 'nivasa'), ('KAG', 'pradhana'), ('KAG', 'mulbery'), ('KAG', 'vikasa'), ('KAG', 'mtonnes'), ('KAG', 'geograpical'), ('KAG', 'anganawadis'), ('KAG', 'vacanthouses'), ('KAG', 'vehiclea'), ('KAG', 'kutirajyoti'), ('KAG', 'gddp'), ('KAG', 'raddish'), ('KAG', 'pwss'), ('KAG', 'bhagyalakshmi'), ('KAG', 'carriag'), ('KAG', 'socie'), ('KAG', 'tubewell'), ('KAG', 'beneficiries'), ('KAG', 'bufaloes'), ('KAG', 'bottel'), ('KAG', 'sericulturist'), ('KAG', 'manaswini'), ('KAG', 'fishlets'), ('KAG', 'bhagyajyoti'

In [72]:
miss_added_set = list(set([item for sublist in miss_added for item in sublist]))
print(len(miss_added_set), miss_added_set)

85 ['suraksha', 'gky', 'nddp', 'gddp', 'panchayth', 'pwss', 'kutirajyoti', 'ksdc', 'brambedkar', 'socie', 'uncultivatable', 'lepros', 'nivasa', 'matsyashraya', 'arecanuts', 'yojane', 'assoistance', 'anganawadis', 'poulation', 'pomogranate', 'swachh', 'vehiclea', 'tubewell', 'hoblies', 'phisical', 'hepatities', 'bhagyajyoti', 'sericulturist', 'sapota', 'bufaloes', 'pmay', 'mulbery', 'pradhana', 'tubectomy', 'oranage', 'avare', 'oftotal', 'estp', 'hectar', 'vikasa', 'anthyodaya', 'vacanthouses', 'sqkms', 'marketin', 'anganawadi', 'kscard', 'geograpical', 'kaccha', 'autonom', 'priva', 'sterlisation', 'areaunder', 'handpump', 'mahathma', 'delivaries', 'madilu', 'multiaxled', 'airculated', 'bhagyalakshmi', 'borewells', 'sinchayee', 'ccusers', 'nrlm', 'employement', 'raddish', 'allopathy', 'mgnrega', 'teachersal', 'carriag', 'fishlets', 'sectorwise', 'lpcd', 'sesamum', 'mythri', 'socities', 'beneficiries', 'rsin', 'manaswini', 'vajapayee', 'raingauge', 'bottel', 'cmkky', 'householdsdemanded'

In [73]:
sdg_count_threshold = sdg_sem.copy()

In [74]:
min_val_list, top_n_index_list = find_similarity_semantic(sdg_count_threshold, new_df, 10)

In [75]:
sdg_count_threshold['top_n_index'] = top_n_index_list
sdg_count_threshold['min_values'] = min_val_list
sdg_count_threshold

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec,top_n_index,min_values
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [...","[[0.004565000534057617, 0.3802250027656555, 0....","[1366, 1367, 1368, 1369, 1370, 1371, 1372, 21,...","[0.8735124280076773, 0.5514384505494149, 0.512..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv...","[[0.07902299799025059, 0.040918700862675905, -...","[380, 381, 455, 456, 510, 858, 859, 860, 861, ...","[0.8411997113893511, 0.8556416402619208, 0.566..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h...","[[-0.13508999347686768, 0.3590700030326843, 0....","[352, 353, 354, 1393, 1394, 1414, 967, 968, 96...","[0.7045711183911056, 0.6798916546022343, 0.527..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l...","[[-0.15599000453948975, -0.4886299967765808, -...","[1013, 1014, 1016, 1017, 1021, 1022, 1023, 102...","[1.0000000000000002, 0.4491569151487498, 0.507..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge...","[[-0.4389500021934509, 0.47641998529434204, 0....","[1034, 1038, 1042, 1055, 1058, 1063, 1073, 108...","[0.6183490921829581, 0.4247187957850606, 0.400..."
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m...","[[0.2156050056219101, -0.03203999996185303, 0....","[18, 1002, 1068, 1078, 1386, 1387, 1388, 1389,...","[0.5807085016601672, 0.5270143992244825, 1.000..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,"[[modern, energy], [ensure], [affordable, reli...","[[0.4583650156855583, -0.1715994980186224, -0....","[1001, 1002, 1003, 1004, 1005, 1006, 1007, 140...","[0.8285528169164933, 0.5270143992244825, 0.471..."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,"[[full, productive], [promote, sustained], [de...","[[-0.29617999494075775, 0.21565200434997678, -...","[903, 895, 1366, 1367, 1368, 1369, 1370, 1371,...","[0.5268596263019111, 0.5479780674858141, 0.703..."
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...,"[[promote], [inclusive, sustainable], [foster,...","[[0.0451820008456707, 0.06997399777173996, -0....","[1061, 1062, 1063, 1064, 1065, 1066, 1067, 106...","[0.5658785348581976, 0.4676713963912552, 0.526..."
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state,"[[reduce], [inequality], [state]]","[[0.21908999979496002, 0.35475999116897583, 0....","[39, 43, 47, 51, 55, 59, 456, 540, 557, 799, 1...","[0.5327591167446397, 0.5327294572224268, 1.000..."


In [76]:
sdg_min_value = sdg_count_threshold.copy()

In [77]:
sdg_count_threshold_all = finding_attr_details(sdg_count_threshold, new_df)
sdg_count_threshold_all

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec,top_n_index,min_values,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [...","[[0.004565000534057617, 0.3802250027656555, 0....","[1366, 1367, 1368, 1369, 1370, 1371, 1372, 21,...","[0.8735124280076773, 0.5514384505494149, 0.512...",40,"[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[14, 14, 14, 14, 14, 14, 14, 1, 1, 1, 1, 1, 1,...","[14.1, 14.1, 14.1, 14.1, 14.1, 14.1, 14.1, 1.6...","[Ddu-gky,Nrlm National Rural Livelihood Missio...","[Rural Development and Panchayat Raj, Rural De...",[Skill Development and Self Employment at the ...
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv...","[[0.07902299799025059, 0.040918700862675905, -...","[380, 381, 455, 456, 510, 858, 859, 860, 861, ...","[0.8411997113893511, 0.8556416402619208, 0.566...",60,"[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[4, 4, 4, 4, 4, 6, 6, 6, 6, 17, 17, 17, 17, 17...","[4.3, 4.3, 4.6, 4.6, 4.12, 6.2, 6.2, 6.2, 6.2,...","[Total Food Grains,Pulses, Area Under Principa...","[Agriculture, Horticulture & Sericulture, Agri...",[Area Under Principal crops in Hectares 2018-1...
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h...","[[-0.13508999347686768, 0.3590700030326843, 0....","[352, 353, 354, 1393, 1394, 1414, 967, 968, 96...","[0.7045711183911056, 0.6798916546022343, 0.527...",48,"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[4, 4, 4, 14, 14, 15, 9, 9, 9, 9, 14, 14, 14, ...","[4.2, 4.2, 4.2, 14.5, 14.5, 15.3, 9.1, 9.1, 9....","[Nos,Tube wells,Gross Net Area Irrigated Hecta...","[Agriculture, Horticulture & Sericulture, Agri...",[Gross and Net area Irrigated under different ...
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l...","[[-0.15599000453948975, -0.4886299967765808, -...","[1013, 1014, 1016, 1017, 1021, 1022, 1023, 102...","[1.0000000000000002, 0.4491569151487498, 0.507...",41,"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 14, 1...","[10.1, 10.1, 10.1, 10.1, 10.1, 10.2, 10.2, 10....","[Male,Rural, Literates 2011 Census, Education,...","[Education, Education, Education, Education, E...","[Literates 2011 Census, Literates 2011 Census,..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge...","[[-0.4389500021934509, 0.47641998529434204, 0....","[1034, 1038, 1042, 1055, 1058, 1063, 1073, 108...","[0.6183490921829581, 0.4247187957850606, 0.400...",46,"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1, 2,...","[10.3, 10.4, 10.5, 10.8, 10.8, 10.9, 10.10, 10...","[Girls,Total Enrolment, No.of Lower Primary Sc...","[Education, Education, Education, Education, E...",[No.of Lower Primary Schools From 1st To 5th a...
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m...","[[0.2156050056219101, -0.03203999996185303, 0....","[18, 1002, 1068, 1078, 1386, 1387, 1388, 1389,...","[0.5807085016601672, 0.5270143992244825, 1.000...",52,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1, 9, 10, 10, 14, 14, 14, 14, 14, 14, 14, 14,...","

In [78]:
sdg_copy = sdg_sem.copy()

In [79]:
sdg_count_threshold_all_goals = process_result_semantic(sdg_count_threshold_all, sdg_copy)

In [80]:
sdg_count_threshold_all.to_csv(absolute_path_semantic, sep=";", index=False)

In [81]:
sdg_count_threshold_all_goals.to_csv(absolute_path_semantic_unroll, sep=";", index=False)

In [82]:
sdg_count_threshold_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,top_n_goal_id
0,1376,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Ddu-gky,Nrlm National Rural Livelihood Mission...",End poverty in all its forms everywhere,1
1,1377,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Cif Community Investment Fund to groups,Nrlm N...",End poverty in all its forms everywhere,1
2,1378,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,Employment through skill training placement Es...,End poverty in all its forms everywhere,1
3,1379,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Persons Skilled,Self Employment Programme,Empl...",End poverty in all its forms everywhere,1
4,1380,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Groups Skilled,Self Employment Programme,Emplo...",End poverty in all its forms everywhere,1
...,...,...,...,...,...,...,...,...
741,1028,10,Education,10.1,Literates 2011 Census,"Total,Urban, Literates 2011 Census, Education","Data, monitoring and accountability",17
742,1029,10,Education,10.1,Literates 2011 Census,"Male,Total, Literates 2011 Census, Education","Data, monitoring and accountability",17
743,1030,10,Education,10.1,Literates 2011 Census,"Female,Total, Literates 2011 Census, Education","Data, monitoring and accountability",17
744,1031,10,Education,10.1,Literates 2011 Census,"Total, Literates 2011 Census, Education","Data, monitoring and accountability",17


In [83]:
sdg_count_sim = sdg_sem.copy()

In [84]:
top_index = find_similarity_semantic_threshold(sdg_min_value, new_df, 0.025)

In [82]:
sdg_count_sim['top_n_index'] = top_index

In [83]:
sdg_count_sim_all = finding_attr_details(sdg_count_sim, new_df)
sdg_count_sim_all

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec,top_n_index,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [...","[[0.004565000534057617, 0.3802250027656555, 0....","[1366, 1367, 1368, 1369, 1370, 1371, 1372, 819...",200,"[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[14, 14, 14, 14, 14, 14, 14, 5, 5, 3, 3, 3, 5,...","[14.1, 14.1, 14.1, 14.1, 14.1, 14.1, 14.1, 5.4...","[Ddu-gky,Nrlm National Rural Livelihood Missio...","[Rural Development and Panchayat Raj, Rural De...",[Skill Development and Self Employment at the ...
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv...","[[0.07902299799025059, 0.040918700862675905, -...","[380, 381, 455, 456, 510, 858, 859, 860, 861, ...",627,"[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[4, 4, 4, 4, 4, 6, 6, 6, 6, 17, 17, 17, 17, 17...","[4.3, 4.3, 4.6, 4.6, 4.12, 6.2, 6.2, 6.2, 6.2,...","[Total Food Grains,Pulses, Area Under Principa...","[Agriculture, Horticulture & Sericulture, Agri...",[Area Under Principal crops in Hectares 2018-1...
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h...","[[-0.13508999347686768, 0.3590700030326843, 0....","[352, 353, 354, 1393, 1394, 1414, 967, 968, 96...",236,"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[4, 4, 4, 14, 14, 15, 9, 9, 9, 9, 9, 14, 14, 1...","[4.2, 4.2, 4.2, 14.5, 14.5, 15.3, 9.1, 9.1, 9....","[Nos,Tube wells,Gross Net Area Irrigated Hecta...","[Agriculture, Horticulture & Sericulture, Agri...",[Gross and Net area Irrigated under different ...
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l...","[[-0.15599000453948975, -0.4886299967765808, -...","[1013, 1014, 1016, 1017, 1021, 1022, 1023, 102...",219,"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[10.1, 10.1, 10.1, 10.1, 10.1, 10.2, 10.2, 10....","[Male,Rural, Literates 2011 Census, Education,...","[Education, Education, Education, Education, E...","[Literates 2011 Census, Literates 2011 Census,..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge...","[[-0.4389500021934509, 0.47641998529434204, 0....","[1034, 1038, 1042, 1055, 1058, 1063, 1073, 108...",465,"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[10.3, 10.4, 10.5, 10.8, 10.8, 10.9, 10.10, 10...","[Girls,Total Enrolment, No.of Lower Primary Sc...","[Education, Education, Education, Education, E...",[No.of Lower Primary Schools From 1st To 5th a...
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m...","[[0.2156050056219101, -0.03203999996185303, 0....","[18, 1002, 1068, 1078, 1386, 1387, 1388, 1389,...",248,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1, 9, 10, 10, 14, 14, 14, 14, 14, 14, 14, 14,...","[1.5, 9.3, 10.9, 10.10, 14.4, 14.4, 14.4, 14.4...","[Working Fire Brigade Water Tankers, No. of Fi...","[General Information, Transport and Communicat...","[No. of Fire Stations on 31-3-2020, Length of ..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",en

In [84]:
sdg_copy2 = sdg_sem.copy()

In [85]:
sdg_count_sim_all_goals = process_result_semantic(sdg_count_sim_all, sdg_copy2)

In [86]:
sdg_count_sim_all.to_csv(absolute_path_sem_threshold, sep=";", index=False)

In [87]:
sdg_count_sim_all_goals.to_csv(absolute_path_sem_threshold_unroll, sep=";", index=False)

In [88]:
sdg_count_sim_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,top_n_goal_id
0,1376,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Ddu-gky,Nrlm National Rural Livelihood Mission...",End poverty in all its forms everywhere,1
1,1377,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Cif Community Investment Fund to groups,Nrlm N...",End poverty in all its forms everywhere,1
2,1378,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,Employment through skill training placement Es...,End poverty in all its forms everywhere,1
3,1379,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Persons Skilled,Self Employment Programme,Empl...",End poverty in all its forms everywhere,1
4,1380,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Groups Skilled,Self Employment Programme,Emplo...",End poverty in all its forms everywhere,1
...,...,...,...,...,...,...,...,...
4288,1134,10,Education,10.13,No.of Medical Colleges and Students 2019-20,"Total,Private,Dental Colleges, No.of Medical C...","Data, monitoring and accountability",17
4289,1140,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17
4290,1141,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Girls,Students from 1st year to final year,Pri...","Data, monitoring and accountability",17
4291,1148,10,Education,10.15,No.of Engineering Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17
